# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [23]:
# try this project here!
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

W0805 09:02:45.674660 140285056018240 hook.py:98] Torch was already hooked... skipping hooking process
W0805 09:02:45.676166 140285056018240 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0805 09:02:45.677012 140285056018240 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0805 09:02:45.677806 140285056018240 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0805 09:02:45.678481 140285056018240 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0805 09:02:45.679307 140285056018240 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0805 09:02:45.680258 140285056018240 base.py:628] Worker bob already exists. Replacing 

In [3]:
model = nn.Linear(2,1)

bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [4]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    alices_loss

In [5]:
# send models to a secure worker
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [6]:
# average
with th.no_grad():
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

In [7]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0176) Alice:tensor(0.0029)
Bob:tensor(0.0151) Alice:tensor(0.0017)
Bob:tensor(0.0125) Alice:tensor(0.0010)
Bob:tensor(0.0102) Alice:tensor(0.0006)
Bob:tensor(0.0082) Alice:tensor(0.0004)
Bob:tensor(0.0065) Alice:tensor(0.0002)
Bob:tensor(0.0051) Alice:tensor(0.0002)
Bob:tensor(0.0040) Alice:tensor(0.0001)
Bob:tensor(0.0031) Alice:tensor(8.2793e-05)
Bob:tensor(0.0024) Alice:tensor(5.9899e-05)


In [8]:
# Check final training results
preds = model(data)
loss = ((preds - target) ** 2).sum()

print(preds)
print(target)
print(loss.data)

tensor([[0.1390],
        [0.1141],
        [0.8560],
        [0.8312]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0816)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [13]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [14]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [15]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [16]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [17]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [18]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [19]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [27]:
import torch
import random

Q = 941083987

In [28]:
# try this project here!


def encrypt(x, n=1):
    if n == 0:
        return ()
    shares = list()
    for i in range(n - 1):
        shares.append(random.randint(0, Q))
        
    final_share = Q - (sum(shares) % Q) + x
    
    shares.append(final_share)
    
    return tuple(shares)

In [29]:
def decrypt(shares):
    return sum(shares) % Q

In [34]:
def add(t1, t2):
    c = list()
    
    for i in range (len(t1)):
        c.append((t1[i] + t2[i]) % Q)
        
    return tuple(c)

In [35]:
decrypt(add(encrypt(5), encrypt(10)))

15

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [36]:
BASE=10
PRECISION=4

In [37]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [38]:
encode(3.5)

35000

In [39]:
decode(35000)

3.5

In [40]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [41]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [42]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [43]:
x = x.share(bob, alice, secure_worker)

In [44]:
bob._objects

{36413730558: tensor([3950212221899534317, 3695402247151981313, 1440209994375886520,
         2560692027858084968, 3420220452083885197])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [45]:
y = x + x

In [46]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:66221944549 -> bob:67959183778]
	-> (Wrapper)>[PointerTensor | me:79508624800 -> alice:15239380602]
	-> (Wrapper)>[PointerTensor | me:84294824852 -> secure_worker:90034079980]
	*crypto provider: me*

In [47]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [48]:
x = th.tensor([0.1,0.2,0.3])

In [49]:
x

tensor([0.1000, 0.2000, 0.3000])

In [50]:
x = x.fix_prec()

In [51]:
x.child.child

tensor([100, 200, 300])

In [52]:
y = x + x

In [53]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [54]:
x = th.tensor([0.1, 0.2, 0.3])

In [55]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [56]:
y = x + x

In [57]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation

In [31]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

model = nn.Linear(2,1)

In [5]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get()

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get()

    bobs_weights = bobs_model.weight.data.clone().get().fix_prec().share(alice, bob)
    bobs_bias = bobs_model.bias.data.clone().get().fix_prec().share(alice, bob)

    alices_weights = alices_model.weight.data.clone().get().fix_prec().share(alice, bob)
    alices_bias = alices_model.bias.data.clone().get().fix_prec().share(alice, bob)
    
    with torch.no_grad():

        model.weight.set_(((bobs_weights + alices_weights) / 2).get().float_prec())
        model.bias.set_(((alices_bias + bobs_bias) / 2).get().float_prec())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0002, requires_grad=True) Alice:tensor(0.0009, requires_grad=True)
Bob:tensor(9.9623e-05, requires_grad=True) Alice:tensor(0.0008, requires_grad=True)
Bob:tensor(5.8365e-05, requires_grad=True) Alice:tensor(0.0007, requires_grad=True)
Bob:tensor(3.5568e-05, requires_grad=True) Alice:tensor(0.0006, requires_grad=True)
Bob:tensor(2.0498e-05, requires_grad=True) Alice:tensor(0.0005, requires_grad=True)
Bob:tensor(1.5027e-05, requires_grad=True) Alice:tensor(0.0003, requires_grad=True)
Bob:tensor(1.0404e-05, requires_grad=True) Alice:tensor(0.0003, requires_grad=True)
Bob:tensor(7.9182e-06, requires_grad=True) Alice:tensor(0.0002, requires_grad=True)
Bob:tensor(6.3605e-06, requires_grad=True) Alice:tensor(0.0002, requires_grad=True)
Bob:tensor(4.9734e-06, requires_grad=True) Alice:tensor(0.0001, requires_grad=True)


In [7]:
# Check final training results
bobs_weights = bobs_model.weight.data.clone().get().fix_prec().share(alice, bob)
bobs_bias = bobs_model.bias.data.clone().get().fix_prec().share(alice, bob)

alices_weights = alices_model.weight.data.clone().get().fix_prec().share(alice, bob)
alices_bias = alices_model.bias.data.clone().get().fix_prec().share(alice, bob)

weights = ((bobs_weights + alices_weights) / 2).get().float_prec()
bias = ((alices_bias + bobs_bias) / 2).get().float_prec()

print(weights)
print(bias)

tensor([[-0.0050,  0.9340]])
tensor([0.0320])
